In [11]:
from dotenv import load_dotenv
import os
from neo4j import GraphDatabase
import pandas as pd

#from langchain_community.graphs import Neo4jGraph

# Warning control
import warnings
warnings.filterwarnings("ignore")

### Neo4j Credentials

In [13]:
%load_ext dotenv
%dotenv

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [18]:
NEO4J_URI = os.getenv('NEO4J_URI')
NEO4J_USERNAME = os.getenv('NEO4J_USERNAME')
NEO4J_PASSWORD = os.getenv('NEO4J_PASSWORD')

### Connecting to Neo4j

In [19]:
URI = NEO4J_URI
AUTH = (NEO4J_USERNAME, NEO4J_PASSWORD)

In [21]:
with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.verify_connectivity()

In [14]:
raw_data = pd.read_csv('./data/char_relations.csv')
raw_data.head()

,name,friend,classmate,teacher,family,immediate family,lover,opponent,colleague,teammate,enemy,"Harry's affection to him (-10 to 10, 0 is the initial value)","Harry's familiarity with him (0 to 10, 0 is the initial value)","His affection to Harry (0 to 10, 0 is the initial value)","His familiarity with Harry (0 to 10, 0 is the initial value)"
0,James,0,0,0,1,1,0,0,0,0,0,0,0,10,0
1,Lily,0,0,0,1,1,0,0,0,0,0,0,0,10,0
2,Petunia,0,0,0,1,0,0,0,0,0,0,0,0,-2,1
3,Vernon Dursley,0,0,0,1,0,0,0,0,0,0,0,0,-2,1
4,Dudley,0,0,0,0,0,0,0,0,0,0,0,0,0,0


### Creating Knowledge Graph

In [22]:
def create_characters(tx, name):
    tx.run("MERGE (p:Person {name: $name})", name=name)

In [61]:
def create_relationship(tx, person1, relationship, person2, attributes):
    safe_relationship = relationship.replace(" ", "_").upper()

    query = (
        f"MATCH (a:Person {{name: $person1}}), (b:Person {{name: $person2}}) "
        f"MERGE (a)-[r:{safe_relationship}]->(b) "
        "SET r += $attributes"
    )
    
    tx.run(query, person1=person1, person2=person2, attributes=attributes)


In [62]:
with driver.session() as session:
    with open('./data/char_relations.csv', 'r') as file:
        
        session.write_transaction(create_characters, 'Harry')
        
        titles = next(file)
        
        titles = titles.strip().split(',')[1:-4]
        
        titles = [title.upper() for title in titles]
        
        for line in file:
            
            name = line.strip().split(',')[0]

            relationships = line.strip().split(',')[1:-4]

            # Create nodes
            session.write_transaction(create_characters, name)

            # Create relationships
            relations = dict(zip(titles, relationships))
            
            existing_relations = [key for key, value in relations.items() if int(value) == 1]
            
            for existing_relation in existing_relations:
                session.write_transaction(create_relationship, 'Harry', existing_relation, name, {})
                session.write_transaction(create_relationship, name, existing_relation, 'Harry', {})

driver.close()